# Classifier
This is more of a first configuration just to try out and make sure it is up and running.
1. Convolutional layer (unit strides) + ReLu
2. Pooling layer
- Convolutional layer (unit strides) + ReLu
- Pooling layer
- Fully connected layer with dropout
- Fully connected layer
- Softmax activation

My knowledge on the best practices of neural networks is not extensive and I'd appreciate some feedback. I referred to [this blog post](https://dev.to/kasperfred/putting-it-all-together---introduction-to-tensorflow-part-5-ein) and this [official page](https://www.tensorflow.org/tutorials/layers).

In [2]:
import tensorflow as tf
import pickle
import numpy as np

In [3]:
with open('dataset.npys', 'rb') as handle:
    train, train_labels, test, test_labels = pickle.load(handle)

In [22]:
with tf.name_scope('input') as scope:
    x = tf.placeholder(tf.float32, [None, 50, 50, 3], name='input')
    labels = tf.placeholder(tf.float32, [None, 2], name='label')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

A side note, to be able to tell what size is produced out of a convolutional layer, with zero-padding on the edges, non-unit stride, and following this [awesome guide](https://arxiv.org/abs/1603.07285v1):
$$o_c = \frac{i + 2p -k}{s}+1$$
where $i$ is the size of the (square) input, $p$ is the amount of zero-padding, $k$ is the size of the (square) kernel and $s$ is the stride. For the pooling layer, the resulting size is described by:
$$o_p = \frac{i-k}{s}+1$$

In [30]:
with tf.name_scope('model') as scope:

    with tf.name_scope('convpool1') as scope:
        conv1 = tf.layers.conv2d(
            inputs = x,
            filters = 32,
            kernel_size = [6, 6],
            padding = 'same',
            activation = tf.nn.relu,
        )
        
        pool1 = tf.layers.max_pooling2d(
            inputs=conv1,
            pool_size=[2,2],
            strides=2
        )
        
    with tf.name_scope('convpool2') as scope:
        conv2 = tf.layers.conv2d(
            inputs=pool1,
            filters=64,
            kernel_size=[3,3],
            padding='same',
            activation=tf.nn.relu
        )
        pool2 = tf.layers.max_pooling2d(
            inputs=conv2,
            pool_size=[3,3],
            strides=1
        )
        
    with tf.name_scope('fc1') as scope:
        pool2_flat = tf.reshape(pool2, [-1, 23 * 23 * 64])
        dense = tf.layers.dense(inputs=pool2_flat, units=516, activation=tf.nn.relu)
        dropout = tf.nn.dropout(dense, keep_prob)
        
    with tf.name_scope('logits') as scope:
        logits = tf.layers.dense(inputs=dropout, units=2)

In [24]:
with tf.name_scope('train') as scope:
    with tf.name_scope('loss') as scope:
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=train_y, logits=logits)
    training_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

with tf.name_scope('evaluation') as scope:
    correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(train_y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# create a summarizer that summarizes loss and accuracy
tf.summary.scalar("Accuracy", accuracy)
tf.summary.scalar("Loss", tf.reduce_mean(cross_entropy))
summary_op = tf.summary.merge_all()

# create saver object
saver = tf.train.Saver()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [ ]:
with tf.Session() as sess:
    # initialize variables
    tf.global_variables_initializer().run()

    # initialize summarizer filewriter
    fw = tf.summary.FileWriter("./summary", sess.graph)

    # train the network
    for step in range(20000):
        
        # this should be a batch instead
        if step%2 == 0:
            acc = sess.run(accuracy, feed_dict={
                x: train, labels: train_labels, keep_prob:1})
            print("mid train accuracy:", acc, "at step:", step)

        if step%100 == 0:
            # compute summary using test data every 100 steps
            summary = sess.run(summary_op, feed_dict={
                x: test, labels: test_labels, keep_prob:1})

            # add merged summaries to filewriter,
            # so they are saved to disk
            fw.add_summary(summary, step)

    # save trained model
    saver.save(sess, "./my.model")